# VROOM testing without ORS
updated with vroom 1.8.0 vroom-express 0.7, supporting Shipments

docs:  https://github.com/VROOM-Project/vroom/blob/v1.8.0/docs/API.md

In [9]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint
from datetime import datetime

# DATI posizione consegne
pointsA = json.load(open('../data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('../data/mezzocorona.geojson'))['features'] #6 points

vroom_url = 'http://vroom:3000/optimization/'

#centro mappa
center = [46.2155, 11.1065]
zoom = 15

# POSIZIONI PARTENZA DEI VEICOLI
point_start = [46.220, 11.106]

# POSIZIONE ARRIVO DEI VEICOLI(uguale a partenza)
point_end = [point_start[0], point_start[1]]

ccA = [p['geometry']['coordinates'] for p in pointsA]
ccB = [p['geometry']['coordinates'] for p in pointsB]

#ELENCO consegne 4 Mezzolombardo e 6 Mezzocorona TOTALE 10 con quantita'
job_locs = ccA + ccB

#colori veicoli/percorsi by vid
Vcolors = {
    '1': 'blue',
    '2': 'red',
    '3': 'purple'
}

Map = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

#convert coordinates (lat,lon) to (lon,lat)
def revcc(cc):
    return list(reversed(cc))

def toKm(m):
    return str(round(m/1000))+'km'

def toMin(sec):
    return str(round(sec/60))+'min'

def toDate(sec):
    return datetime.fromtimestamp(sec).strftime("%H:%M")

def vehicle2Map(vid, cap, loc, timeWindow=[]):
    if timeWindow is None: 
        timeWindow = []
    
    mText = "vehicle "+str(vid)+"<br> capacity %s" % cap
    
    if len(timeWindow) > 0:
        mText += "<br>times from: "+str(toDate(timeWindow[0]))+"<br>to "+str(toDate(timeWindow[1]))
    
    mColor = Vcolors[str(vid)]
    mark = folium.Marker(loc, tooltip=folium.Tooltip(mText,permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))
    mark.add_to(Map)

vehicles = []

#veicolo 1
vid = 1
capacity = 1

timeWindow = [          #orario lavoro del veicolo
        1612880173,  #dalle 15:16
        1612980273   #alle 19:04
    ]

loc = [point_start[0] + 0.004, point_start[1] + 0.004]
vehicles.append({
    'id': vid,
    'capacity': [capacity],  # Limite di capacita' del Veicolo    
    'profile': 'driving-car',
    'start': revcc(loc),     # posizione di partenza veicoli
    'end': revcc(loc),
    'time_window': timeWindow,
    'breaks': [              #orari pause sanitizzazione del veicolo
        {
          "id": 1,
          "description": "pausa sanitizazione del veicolo", #descrizione pausa
          "service": 300,   #durata pausa 5 minuti
          "time_windows": [ #an array of time_window objects describing valid slots for break start
            [
              1612881173, #dalle 15:32
              1612882173  #alle 15:49
            ],
            [
              1612884173, #dalle 16:22
              1612885173  #alle 16:39
            ]
          ]
        }
      ]
})
vehicle2Map(vid, capacity, loc, timeWindow)

#veicolo 2
vid = 2
capacity = 1

timeWindow = [          #orario lavoro del veicolo
        1612880173,  #dalle 15:16
        1612980273   #alle 19:04
    ]

loc = [point_start[0] + 0.003, point_start[1] + 0.003]
vehicles.append({
    'id': vid,
    'capacity': [capacity],  # Limite di capacita' del Veicolo    
    'profile': 'driving-car',
    'start': revcc(loc), # posizione di partenza veicoli
    'end': revcc(loc),
    'time_window': timeWindow
})
vehicle2Map(vid, capacity, loc, timeWindow)


#DEFINTION OF JOBS

jobs = []
for idx, cc in enumerate(job_locs):
    loc = revcc(cc)
    
    #quantita da consegnare
    delivery = 1 #random.randrange(1, 3)
    
    #tempo di della singola consegna(in secondi)
    service = random.randrange(60, 300) #da 2 a 5 minuti
    
    timeWindow = [         #orario di consegna del paziente
            1612880173,  #dalle 15:16
            1612980273   #alle 19:04
            #1612883173, #dalle 16:06
            #1612883973  #alle 16:19
        ]
    
    # shifting casuale del tempo per ogni job
    s = random.randrange(-3000,3000)
    timeWindow = [timeWindow[0]+s, timeWindow[1]+s]
    
    jobs.append({
        'id': idx,
        'location': cc,
        'delivery': [delivery], # Quantita' da consegnare
        #'service': service,      # tempo di consegna
        #'time_windows': [
        #    timeWindow
        #]
    })
    
    mText = "pack "+str(idx)+"<br> quantity %s <br> delivery time %s" % (delivery, toMin(service))
    
    if len(timeWindow) > 0:
        mText += "<br>times from: "+str(toDate(timeWindow[0]))+"<br>to "+str(toDate(timeWindow[1]))
    
    mark = folium.Marker(loc, tooltip=folium.Tooltip(mText, permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

Map

In [10]:
request = {
    'vehicles': vehicles,
    'jobs': jobs,
    'geometry': True
}

request

{'vehicles': [{'id': 1,
   'capacity': [1],
   'profile': 'driving-car',
   'start': [11.11, 46.224],
   'end': [11.11, 46.224],
   'time_window': [1612880173, 1612980273],
   'breaks': [{'id': 1,
     'description': 'pausa sanitizazione del veicolo',
     'service': 300,
     'time_windows': [[1612881173, 1612882173], [1612884173, 1612885173]]}]},
  {'id': 2,
   'capacity': [1],
   'profile': 'driving-car',
   'start': [11.109, 46.223],
   'end': [11.109, 46.223],
   'time_window': [1612880173, 1612980273]}],
 'jobs': [{'id': 0,
   'location': [11.092028617858887, 46.2183172744161],
   'delivery': [1]},
  {'id': 1,
   'location': [11.08793020248413, 46.2161294935962],
   'delivery': [1]},
  {'id': 2,
   'location': [11.091685295104979, 46.21057627744751],
   'delivery': [1]},
  {'id': 3,
   'location': [11.096470355987549, 46.21344204466888],
   'delivery': [1]},
  {'id': 4,
   'location': [11.114227983093262, 46.21858452155989],
   'delivery': [1]},
  {'id': 5,
   'location': [11.118

In [15]:
resp = requests.post(vroom_url, json = request)

solution = resp.json()

#solution['summary']
#solution

In [16]:
if len(solution['routes']) > 0:
    for route in solution['routes']:

        rtext = 'distance: '+toKm(route['distance'])+' delivered quantity: '+str(route['delivery'][0])+', time: '+toMin(route['duration']+route['service'])
        rcolor = Vcolors[str(route['vehicle'])]
        print('Vehicle'+str(route['vehicle']), rcolor, rtext)

        folium.PolyLine(
            locations = [revcc(coords) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],
            color = rcolor,
            opacity = 0.8,
            weight = 5
        ).add_to(Map)

    print("\nTotal distance: "+toKm(solution['summary']['distance']), 'Not Delivered: '+str(len(solution['unassigned'])))

Map

Vehicle1 blue distance: 4km delivered quantity: 1, time: 11min
Vehicle2 red distance: 3km delivered quantity: 1, time: 5min

Total distance: 6km Not Delivered: 8


In [13]:
solution

{'code': 0,
 'summary': {'cost': 664,
  'unassigned': 8,
  'delivery': [2],
  'amount': [2],
  'pickup': [0],
  'service': 300,
  'duration': 664,
  'waiting_time': 0,
  'priority': 0,
  'distance': 6311,
  'computing_times': {'loading': 33, 'solving': 1, 'routing': 23}},
 'unassigned': [{'id': 9, 'location': [11.114301681518556, 46.20900226913922]},
  {'id': 8, 'location': [11.122541427612305, 46.20986352455638]},
  {'id': 7, 'location': [11.127262115478516, 46.211972058400264]},
  {'id': 6, 'location': [11.123356819152832, 46.21419929496722]},
  {'id': 1, 'location': [11.08793020248413, 46.2161294935962]},
  {'id': 2, 'location': [11.09168529510498, 46.21057627744751]},
  {'id': 3, 'location': [11.096470355987549, 46.21344204466888]},
  {'id': 4, 'location': [11.114227983093262, 46.21858452155989]}],
 'routes': [{'vehicle': 1,
   'cost': 340,
   'delivery': [1],
   'amount': [1],
   'pickup': [0],
   'service': 300,
   'duration': 340,
   'waiting_time': 0,
   'priority': 0,
   'dist